In [1]:
from music21 import *
import os
from fractions import *
import numpy as np
from midi_note import MIDINote
import json

In [2]:
#Global Variables

PATH = "POP909/"
LIMIT = 10
MIDI_NOTE_DIRECTORY = "midi_notes_list.json"
MAX_NOTE_VAL = 125



In [3]:
def parse_raw_notes(file):
    print("Loading Music File:", file)
    raw_notes = []

    midi_data = converter.parse(file)

    for part in midi_data.parts:
        if part.partName == 'MELODY':
            midi_elements = part.recurse()
            for element in midi_elements:
                if isinstance(element, note.Note) or isinstance(element, note.Rest):
                    note_duration = duration.Duration()
                    note_duration.quarterLength = element.quarterLength

                    if isinstance(element, note.Rest):
                        raw_note = str(element.name)
                    else:
                        raw_note = str(element.pitch)

                    raw_notes.append(MIDINote(raw_note, str(note_duration.type), str(element.quarterLength)))

    return np.array(raw_notes)

In [4]:
def get_midi_file_paths():
    path = PATH
    limit = LIMIT
    midi_files_path = []
    for index, i in enumerate(os.listdir(path)):
        if index == limit:
            break
        for j in os.listdir(path + i):
            midi_path = (i + '/' + j)
            if midi_path.endswith(".mid"):
                midi_files_path.append(midi_path)
    return np.array(midi_files_path)

In [5]:
def load_data():
    midi_files_path = get_midi_file_paths()

    raw_notes_list = []

    for midi_path in midi_files_path:
        parsed_midi_notes = parse_raw_notes(PATH + midi_path)
        raw_notes_list.append(parsed_midi_notes)

    return np.array(raw_notes_list).reshape((LIMIT, 1))

In [6]:
def to_json(midi_notes_list):
    json_file = {}
    for index, midi_notes in enumerate(midi_notes_list):
        notes = midi_notes[0]

        json_file[index] = [_note.as_map for _note in notes]

    with open(MIDI_NOTE_DIRECTORY) as f:
        data = json.load(f)
        data.update(json_file)
    with open(MIDI_NOTE_DIRECTORY, 'w') as f:
        json.dump(data, f)

In [17]:
def json_to_midi_notes_list(batch=None):
    """Converts json to numpy array
    Args
        batch: A list/tuple defining the start and end to be converted
    """

    with open(MIDI_NOTE_DIRECTORY) as f:
        data = json.load(f)
    m = len(data)
    midi_notes_list = []

    if batch:
        start, end = batch
        end = min(m-1, end)
        for index in range(start, end+1):
            midi_notes_list.append(np.array(data[str(index)]))
    else:
        for index in range(m):
            midi_notes_list.append(np.array(data[str(index)]))

    return np.array(midi_notes_list)

In [26]:
def map_note(_note):
    """Adjusts note strings to be between the 5th & 6th octaves,
       \nthen maps the adjusted note string to an integer.

       Parameters:
         _note (string): A string representing a note.

       Returns:
        int: An integer representing the note.
    """
    note_letter = _note[:-1]
    note_octave = _note[-1]
    note_octave = int(note_octave)
    note_octave = 5 if note_octave <= 5 else 6
    notes_map = {
        'C': 0,
        'C#': 1,
        'D': 2,
        'D#': 3,
        'E': 4,
        'F': 5,
        'F#': 6,
        'G': 7,
        'G#': 8,
        'A': 9,
        'A#': 10,
        'B': 11
    }
    return notes_map[note_letter] + ((note_octave-5) * 12)

def normalize(mapped_notes):
    """Normalize data using min-max scaling
    Args
        value: Integer from 0-125 note mapping

    """
    return mapped_notes/MAX_NOTE_VAL

def transform_dataset(all_midi_notes_list):
    """Transforms numpy array of midi notes list into model input data for training
    Args
        all_midi_notes_list: list of all midi notes
    """

    dataset = []

    for notes_list in all_midi_notes_list:
        mapped = []
        for _note in notes_list:
            mapped.append(map_note(_note))

        dataset.append(normalize(np.array(mapped)))

    return np.array(dataset)




In [2]:
# Runner
all_midi_notes = load_data()
to_json(all_midi_notes)

NameError: name 'load_data' is not defined

In [ ]:
json_to_midi_notes_list(batch=[0,0])

array([[{'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'quarter', 'length': '1.0'},
        {'note': 'C#4', 'duration_type': '16th', 'length': '0.25'},
        {'note': 'rest', 'duration_type': '32nd', 'length': '1/12'},
        {'note': 'E-4', 'duration_type': '16th', 'length': '0.25'},
        {'note': 'F#4', 'duration_type': '16th', 'length': '0.25'},
        {'note': 'G#4', 'duration_type': '16

0